## 실습 : Model 을 yaml file 로 저장하고 복원 후 weight load 및 compile 및 predict

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

# LeNet
model = tf.keras.Sequential()

model.add(Conv2D(6, kernel_size=5, padding="same", input_shape=(28, 28, 1)))
model.add(Activation("relu"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding="valid"))

model.add(Conv2D(16, kernel_size=5, padding="valid"))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

model.add(Flatten())

model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))

model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, epochs=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test)
print(score[0])
print(score[1])

Instructions for updating:
Colocations handled automatically by placer.
Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
10000/10000 [==============================] - 1s 75us/sample - loss: 0.0747 - acc: 0.9761
0.07467947590909899
0.9761


In [2]:
yaml_string = model.to_yaml()

In [4]:
with open("lenet.yaml", "w") as yaml_file:
    yaml_file.write(yaml_string)

In [9]:
from tensorflow.keras.models import model_from_yaml

yaml_file = open("lenet.yaml", "r")
read_model = yaml_file.read()
yaml_file.close()

loaded_model = model_from_yaml(read_model)
loaded_model.load_weights("lenet_weight.h5")

loaded_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [10]:
score = loaded_model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 87us/sample - loss: 0.0654 - acc: 0.9798


In [14]:
y_predict = model.predict(X_test)

In [15]:
np.argmax(y_predict, axis=1)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)